# INLA-R: Create models and assess predictions
## Models per quartile

In [436]:
#library(rstanarm)
library(INLA)
library(brinla)
library(data.table)
library(ggplot2)
options(repr.plot.width=3, repr.plot.height=3)

In [437]:
# read data
df = fread('../data/le_cov_sel.csv')
nrow(df)

[1] 12000

In [438]:
print(names(df))

 [1] "le"                   "z_relative_mob"       "z_gini"              
 [4] "county"               "gender"               "income_q"            
 [7] "county_name"          "stateabbrv"           "statename"           
[10] "log_population"       "log_income"           "z_segregation_income"
[13] "log_unemployment"     "z_uninsured"          "z_medicare_expenses" 
[16] "log_crime_rate"       "log_pct_black"        "log_pct_hispanic"    
[19] "z_obesity"            "z_smoking"            "z_exercise"          


In [439]:
df[, state := .GRP, by = statename]
df[, cty := .GRP, by = county]
df[, income_qr := .GRP, by = income_q]

In [440]:
table(df[, .(income_qr, income_q)]) # ok, right!

         income_q
income_qr   Q1   Q2   Q3   Q4
        1 3000    0    0    0
        2    0 3000    0    0
        3    0    0 3000    0
        4    0    0    0 3000

In [441]:
df[, z_relative_mob := z_relative_mob * -1.0]

# INLA Models (using PC prior)

In [442]:
# create auxiliary variables
df[, state_mob := state]
df[, state_gini := state]
# df[, cty := county]
# df[, cty_mob := county]
# df[, cty_gini := county]
# df[, q_mob := income_qr]
# df[, q_gini := income_qr]
# df[, q_exercise := income_qr]

In [443]:
female = df[gender=='F']
male = df[gender=='M']

# Baseline model

### Male

In [444]:
# define PC prior
# same prior per quartile
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income, male)

# pc prior
sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [445]:
# run models per income quartile
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
       f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = male[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('m1_', i)
    assign(model_name, model)  
    }

In [446]:
m1_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),36.4221332,0.12504708,36.1765129,36.4217664,36.6693818,36.4210395,0.000000e+00
z_relative_mob,0.4865704,0.04037955,0.4072665,0.4865742,0.5657799,0.4865851,8.511328e-15
z_gini,0.3189742,0.03733456,0.2456581,0.3189748,0.3922189,0.3189791,2.737983e-14
log_population,-0.2542599,0.03939753,-0.3315979,-0.2542694,-0.1769418,-0.2542849,2.458752e-14
log_income,1.1578343,0.18952907,0.7856607,1.1578319,1.5296750,1.1578432,2.472569e-14


In [447]:
bri.hyperpar.summary(m1_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.081663,0.01997705,1.0431700,1.0813494,1.121600,1.0806705
SD for state,0.815244,0.09513868,0.6473607,0.8082699,1.020416,0.7934126


In [448]:
# simulate values
source('utils/simulation_no_random_effects.R')

In [449]:
# create data for prediction
# all values in their means except for constrast: income mobility
nrep = 2 # 4 quartiles for 2 contrast values
relative_mob_pred_data = data.table(
    z_relative_mob = c(0.0, 1.0),
    z_gini = rep(0, nrep),
    log_population = rep(0, nrep), 
    log_income = rep(0, nrep))

In [450]:
# simulate values per quartile
sim_male_m1 = data.table()

for (i in 1:4) {
    model_name = paste0('m1_', i)
    s = simulate_pred_no_re(model=get(model_name), 
                                          data=relative_mob_pred_data, 
                                          contrast='z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(pred)), by = sim][, .(q, fd)]
    sim_male_m1 = rbind(sim_male_m1, d)
    }

In [451]:
fwrite(sim_male_m1, file = '../data/sim_male_m1.csv')

### Female

In [452]:
# define PC prior
# same prior per quartile
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income, female)

sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [453]:
# run models per income quartile
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
       f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = female[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('f1_', i)
    assign(model_name, model)   
    }

In [454]:
f1_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),41.9454230,0.10679946,41.7349004,41.9453582,42.1559921,41.9452359,5.733354e-19
z_relative_mob,0.3462274,0.03967042,0.2683248,0.3462276,0.4240561,0.3462315,2.425041e-14
z_gini,0.2673734,0.03730399,0.1941220,0.2673725,0.3405621,0.2673737,2.742473e-14
log_population,-0.2453674,0.03932509,-0.3225765,-0.2453722,-0.1682035,-0.2453786,1.906952e-14
log_income,0.6489395,0.18938983,0.2770545,0.6489320,1.0205201,0.6489330,2.321444e-14


In [455]:
bri.hyperpar.summary(f1_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.0845787,0.02001332,1.0459740,1.0842817,1.1245466,1.0836591
SD for state,0.6833374,0.08319916,0.5361704,0.6774347,0.8624172,0.6656124


In [456]:
# simulate values per quartile
sim_female_f1 = data.table()

for (i in 1:4) {
    model_name = paste0('f1_', i)
    s = simulate_pred_no_re(model=get(model_name), 
                                          data=relative_mob_pred_data, 
                                          contrast='z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(pred)), by = sim][, .(q, fd)]
    sim_female_f1 = rbind(sim_female_f1, d)
    }

In [457]:
fwrite(sim_female_f1, file = '../data/sim_female_f1.csv')

# Adjusting for contextual variables

### Male

In [458]:
# -Demographic: % Black, Hispanic (since Chetty uses this to race adjust)
# -Social: Crime rate, segregation
# -Economic: Gini, possibly unemployment
# -Health Access: uninsured, medicare expenses

In [459]:
# define PC prior
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income + 
           log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
           log_unemployment +  z_uninsured + z_medicare_expenses, male)


sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [460]:
# models per quartile
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
        log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
        log_unemployment +  z_uninsured + z_medicare_expenses +
        f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = male[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('m2_', i)
    assign(model_name, model)
    
    }

In [461]:
m2_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),36.40819005,0.12772047,36.158663703,36.40731663,36.662393971,36.40558437,6.969295e-12
z_relative_mob,0.39106906,0.04700942,0.298771122,0.39106388,0.483309808,0.39105747,3.139933e-14
z_gini,0.22616923,0.04252941,0.142670304,0.22616357,0.309621789,0.22615583,1.918145e-14
log_population,-0.07253352,0.05576715,-0.181984485,-0.07255371,0.036927413,-0.07258960,1.171366e-14
log_income,1.66689403,0.22158470,1.231762767,1.66689530,2.101613669,1.66691668,2.035041e-14
log_crime_rate,-0.21794802,0.06325331,-0.342135837,-0.21795607,-0.093831179,-0.21796685,2.081157e-14
z_segregation_income,-0.02413704,0.05258630,-0.127427021,-0.02412822,0.079008110,-0.02410604,1.542801e-14
log_pct_black,-0.06721094,0.03084773,-0.127766541,-0.06721784,-0.006673433,-0.06722912,1.936924e-14
log_pct_hispanic,-0.08308247,0.04337491,-0.168291416,-0.08307117,0.001984769,-0.08304475,2.120708e-14
log_unemployment,0.28563296,0.14912758,-0.007283337,0.28565816,0.578137560,0.28572147,1.903286e-14


In [462]:
bri.hyperpar.summary(m2_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.0409372,0.01930077,1.003781,1.0406196,1.079555,1.0399182
SD for state,0.8279269,0.10227067,0.648109,0.8202341,1.049101,0.8040011


In [463]:
# create data for predictions
nrep =  2 # 2 contrast values
relative_mob_pred_data = data.table(
    z_relative_mob       = c(0.0, 1.0),
    z_gini               = rep(0, nrep),
    log_population       = rep(0, nrep), 
    log_income           = rep(0, nrep),
    log_crime_rate       = rep(0, nrep),
#     log_poverty          = rep(0, nrep),
#     log_mig_inflow       = rep(0, nrep),
#     log_mig_outflow      = rep(0, nrep),
#     log_foreign          = rep(0, nrep),
    log_pct_black        = rep(0, nrep),
    log_pct_hispanic     = rep(0, nrep),
#     log_house_value      = rep(0, nrep),
#     log_local_gov_exp    = rep(0, nrep),
    log_unemployment     = rep(0, nrep),
    z_segregation_income = rep(0, nrep),
#     z_religion           = rep(0, nrep),
#     z_labor_force        = rep(0, nrep),
#     z_college            = rep(0, nrep),
#     z_middle_class       = rep(0, nrep),
    z_uninsured          = rep(0, nrep), 
    z_medicare_expenses  = rep(0, nrep))

In [464]:
# simulate by quartile
sim_male_m2 = data.table()

for (i in 1:4) {
    model_name = paste0('m2_', i)
    s = simulate_pred_no_re(model=get(model_name), 
                                          data=relative_mob_pred_data, 
                                          contrast='z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(pred)), by = sim][, .(q, fd)]
    sim_male_m2 = rbind(sim_male_m2, d)
    }

In [465]:
fwrite(sim_male_m2, file = '../data/sim_male_m2.csv')

### Female

In [466]:
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income + 
       log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
       log_unemployment +  z_uninsured + z_medicare_expenses, female)

# pc prior
sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [467]:
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
        log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
        log_unemployment +  z_uninsured + z_medicare_expenses +
        f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = female[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('f2_', i)
    assign(model_name, model)
    
    }

In [468]:
f2_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),41.93323512,0.10655149,41.72344206,41.93307085,4.214361e+01,41.93275641,0.000000e+00
z_relative_mob,0.27786991,0.04701842,0.18554375,0.27786837,3.701187e-01,0.27786927,2.667922e-14
z_gini,0.26144638,0.04252986,0.17793732,0.26144392,3.448914e-01,0.26144258,2.493536e-14
log_population,0.02852776,0.05517583,-0.07981833,0.02852671,1.367789e-01,0.02852927,1.942707e-14
log_income,0.95109118,0.22044950,0.51824029,0.95107463,1.383631e+00,0.95106009,2.113166e-14
log_crime_rate,-0.31948495,0.06325271,-0.44368423,-0.31948863,-1.953808e-01,-0.31949067,2.167913e-14
z_segregation_income,-0.10303745,0.05251211,-0.20616168,-0.10303545,-2.016894e-05,-0.10302696,1.635628e-14
log_pct_black,-0.07427762,0.03053746,-0.13425779,-0.07427313,-1.437811e-02,-0.07426149,2.302023e-14
log_pct_hispanic,-0.20946949,0.04284506,-0.29360687,-0.20946930,-1.254116e-01,-0.20946526,2.740480e-14
log_unemployment,0.51107474,0.14741970,0.22159697,0.51106958,8.003115e-01,0.51107175,2.362712e-14


In [469]:
bri.hyperpar.summary(f2_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.0460784,0.01937325,1.0087380,1.0457785,1.0847973,1.0451351
SD for state,0.6769903,0.08466052,0.5278019,0.6707777,0.8597488,0.6580783


In [470]:
# simulate per quartile
sim_female_f2 = data.table()

for (i in 1:4) {
    model_name = paste0('f2_', i)
    s = simulate_pred_no_re(model=get(model_name), 
                                          data=relative_mob_pred_data, 
                                          contrast='z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(pred)), by = sim][, .(q, fd)]
    sim_female_f2 = rbind(sim_female_f2, d)
    }

In [471]:
fwrite(sim_female_f2, file = '../data/sim_female_f2.csv')

# Table by income quartile

In [472]:
library(texreg)
source('utils/extract_inla.R')

In [473]:
for (i in 1:4) {
    cmodels <- rep(c('Women', 'Men'), 2)
    models <- list(get(paste0('f1_', i)),
                   get(paste0('m1_', i)),
                   get(paste0('f2_', i)),
                   get(paste0('m2_', i)))

    cnames <- list("(Intercept)" = 'Constant',
                   'z_relative_mob' = 'Standardized Income mobility (Rank-Rank Slope x -1)',
                   'z_gini' = 'Standardized Gini',
                   "sd for state" = "SD states",
                   "sd for the Gaussian observations" = "SD observations")

    # screenreg(models)
    t = texreg(models,
                include.dic = TRUE, include.waic = TRUE,
                ci.test = FALSE,
                float.pos = "htp",
                caption = "Life Expectancy (40) Models",
                booktabs = TRUE,
                use.packages = FALSE,
                dcolumn = TRUE,
                caption.above = TRUE,
                scalebox = 0.65,
                label = "inla_models",
                # sideways = TRUE,
                digits = 2,
                custom.model.names = cmodels,
                custom.coef.map = cnames,
                groups = list("Random Effects" = c(4:5)),
                custom.note = "Note: Selected coefficients 
                (mean of marginal posterior distribution). 95\\% credibility intervals.")
    
    assign(paste0('tab_', i), t)
    
}

In [474]:
heading = '\\renewcommand{\\arraystretch}{1.2}\n
\\begin{table}[htp]\n
\\begin{threeparttable}\n
\\caption{Life Expectancy (40) Models\\tnote{1}}\\label{inla_models}\n
\\centering\n
\\setlength{\\tabcolsep}{1pt}\n
\\scriptsize\n
\\begin{tabular}{l D{.}{.}{5.11} D{.}{.}{5.11} D{.}{.}{5.11} D{.}{.}{5.11} }\n
\\toprule\n
& \\multicolumn{2}{c}{Baseline\\tnote{2}} & \\multicolumn{2}{c}{Social Indicators\\tnote{3}} \\\\\n
& \\multicolumn{1}{c}{Women} & \\multicolumn{1}{c}{Men} & \\multicolumn{1}{c}{Women} & \\multicolumn{1}{c}{Men} \\\\\n
\\midrule\n'

In [475]:
heading =  gsub("\n\n", "\n", heading)

In [476]:
bottom = '\\addlinespace[5pt]\n
\\bottomrule\n
\\end{tabular}
\n\\begin{tablenotes}[flushleft]\n
\\scriptsize\n
\\item [1] Four separated models (one per income quartile). Selected coefficients, mean of marginal posterior distribution and 95\\% credibility intervals in brackets.\n
\\item [2] Baseline model adjusts for log population and log income.\n
\\item [3] Social indicators model adjusts for log population, log income, log crime rate, log \\% black, log \\% hispanic, log unemployment, z-score income segregation, z-score \\% uninsured, and z-score medicare expenses.\n\\end{tablenotes}\n\\end{threeparttable}\n
\\end{table}\n'

In [477]:
bottom =  gsub("\n\n", "\n", bottom)

In [478]:
sep = NA
for (i in 1:4) {
  sep[i] = paste0("\\addlinespace[10pt]\n\\multicolumn{5}{l}{\\textbf{Income Quartile ", i, "}} \\\\\n\\addlinespace[10pt]\n")
}

In [479]:
tabs = list(tab_1, tab_2, tab_3, tab_4)

In [480]:
out = list()
for (i in 1:4) {
    out[[i]] = gsub('(.+)(Constant.+)(Random.+)', '\\2', tabs[[i]])
}

In [481]:
# export table
cat(heading, 
    sep[[1]], out[[1]], 
    sep[[2]], out[[2]], 
    sep[[3]], out[[3]], 
    sep[[4]], out[[4]], 
    bottom, 
    file = 'tables/inla_models_quartile.tex')

# Forest plot

In [482]:
models = c('m1_1', 'm1_2', 'm1_3', 'm1_4', 'm2_1', 'm2_2', 'm2_3', 'm2_4')
quartile = rep(c('Q1', 'Q2','Q3', 'Q4'), 2)

coeff = data.frame()

for (i in seq_along(models)) {
    temp = get(models[i])$summary.fixed['z_relative_mob',c('mean', '0.025quant', '0.975quant')]
    temp$quartile = quartile[i]
    temp$model = ifelse(grepl('m1', models[i]), 'Baseline', 'Adjustments')
    coeff = rbind(coeff, temp)
}


setnames(coeff, names(coeff), c('estimate', 'low', 'high', 'quartile', 'model'))
coeff$model = factor(coeff$model, levels=c( 'Adjustments', 'Baseline'))
coeff$quartile = factor(coeff$quartile, levels=c('Q4', 'Q3', 'Q2', 'Q1'))

In [483]:
coeff

,estimate,low,high,quartile,model
z_relative_mob,0.4865704,0.40726649,0.5657799,Q1,Baseline
z_relative_mob1,0.3877516,0.31005973,0.4653413,Q2,Baseline
z_relative_mob2,0.3035801,0.21914448,0.3878838,Q3,Baseline
z_relative_mob3,0.1909537,0.07559253,0.3055712,Q4,Baseline
z_relative_mob4,0.3910691,0.29877112,0.4833098,Q1,Adjustments
z_relative_mob5,0.2265208,0.13792422,0.3150656,Q2,Adjustments
z_relative_mob6,0.1211239,0.02512365,0.2170676,Q3,Adjustments
z_relative_mob7,0.1061787,-0.02875057,0.2405835,Q4,Adjustments


In [484]:
sdazar::savepdf('plots/coeff_male', 12, 10)
ggplot(coeff, aes(colour = model)) + 
       geom_hline(yintercept = 0, colour = gray(1/2), lty = 3) + 
       geom_linerange(aes(x = quartile, ymin = low, ymax = high),
                            lwd = 1, position = position_dodge(width = 1/2)) + 
       geom_pointrange(aes(x = quartile, y = estimate, ymin = low,
                                 ymax = high),
                             lwd = 1/2, position = position_dodge(width = 1/2),
                             shape = 21, fill = "white") +
        coord_flip() + theme_bw() + 
        theme(legend.position = 'top', legend.title = element_blank()) + 
        labs(y='\nCoefficient (95% credibility intervals)', x='') + 
        scale_color_manual(values=c("#ff8080", "#8080ff")) + 
        ylim(-0.2, 0.7)
dev.off()

pdf 
  2

In [485]:
models = c('f1_1', 'f1_2', 'f1_3', 'f1_4', 'f2_1', 'f2_2', 'f2_3', 'f2_4')
coeff = data.frame()

for (i in seq_along(models)) {
    temp = get(models[i])$summary.fixed['z_relative_mob',c('mean', '0.025quant', '0.975quant')]
    temp$quartile = quartile[i]
    temp$model = ifelse(grepl('f1', models[i]), 'Baseline', 'Adjustments')
    coeff = rbind(coeff, temp)
}


setnames(coeff, names(coeff), c('estimate', 'low', 'high', 'quartile', 'model'))
coeff$model = factor(coeff$model, levels=c( 'Adjustments', 'Baseline'))
coeff$quartile = factor(coeff$quartile, levels=c('Q4', 'Q3', 'Q2', 'Q1'))

In [486]:
coeff

,estimate,low,high,quartile,model
z_relative_mob,0.346227375,0.26832481,0.42405606,Q1,Baseline
z_relative_mob1,0.246915125,0.17062197,0.32310945,Q2,Baseline
z_relative_mob2,0.147446179,0.05854679,0.23596087,Q3,Baseline
z_relative_mob3,0.240610986,0.13598961,0.34485057,Q4,Baseline
z_relative_mob4,0.277869915,0.18554375,0.37011874,Q1,Adjustments
z_relative_mob5,0.122199202,0.03083663,0.21348798,Q2,Adjustments
z_relative_mob6,-0.007018605,-0.11217040,0.09798843,Q3,Adjustments
z_relative_mob7,0.087009734,-0.04329048,0.21722320,Q4,Adjustments


In [487]:
sdazar::savepdf('plots/coeff_female', 12, 10)
ggplot(coeff, aes(colour = model)) + 
       geom_hline(yintercept = 0, colour = gray(1/2), lty = 3) + 
       geom_linerange(aes(x = quartile, ymin = low, ymax = high),
                            lwd = 1, position = position_dodge(width = 1/2)) + 
       geom_pointrange(aes(x = quartile, y = estimate, ymin = low,
                                 ymax = high),
                             lwd = 1/2, position = position_dodge(width = 1/2),
                             shape = 21, fill = "white") +
        coord_flip() + theme_bw() + 
        theme(legend.position = 'top', legend.title = element_blank()) + 
        labs(y='\nCoefficient (95% credibility intervals)', x='') + 
        scale_color_manual(values=c("#ff8080", "#8080ff")) +
        ylim(-0.2, 0.7)
dev.off()

pdf 
  2

# Prediction per county

In [488]:
library(sdazar)
library(ggridges)
library(forcats)

In [489]:
# order data by le
setorder(male, le)
setorder(female, le)

In [490]:
# county state labes
df[, cn := paste0(county_name, ", ", stateabbrv)]
cn = df[, .(cn, state, county)]
cn = cn[!duplicated(cn)]
head(cn)

cn,state,county
"Autauga, AL",1,1001
"Baldwin, AL",1,1003
"Barbour, AL",1,1005
"Blount, AL",1,1009
"Calhoun, AL",1,1015
"Chambers, AL",1,1017


In [491]:
# selected variables
vars = c('le', 'county', 'state', 'z_relative_mob', 'z_gini', 'log_population', 'log_income' ,  
        'log_crime_rate', 'z_segregation_income', 'log_pct_black', 'log_pct_hispanic', 
        'log_unemployment', 'z_uninsured', 'z_medicare_expenses')

### Male

In [492]:
max_mob = max(male$z_relative_mob)
max_mob

[1] 3.538464

In [493]:
bottom_male = head(male[income_qr==1, vars, with=FALSE], 10)
bottom_male_c = copy(bottom_male)
bottom_male_c[, z_relative_mob := max_mob]
tot_bottom_male = rbind(bottom_male, bottom_male_c) 
tot_bottom_male[, ranking := .GRP, by = .(county, state)]
tot_bottom_male[, order := seq_len(.N), by = .(county, state)]
anyDuplicated(tot_bottom_male[,.(state, county, z_relative_mob)])

[1] 0

In [494]:
# simulate values
source('utils/simulation_random_intercept.R')

In [495]:
sim_male_bottom = simulate_pred_re(m2_1, nsim=2000, tot_bottom_male, contrast='z_relative_mob', 
                  random_intercept='state')

In [496]:
sim_male_bottom = merge(sim_male_bottom, cn, by = c('county', 'state'))

In [497]:
setorder(sim_male_bottom, ranking)
head(sim_male_bottom)

county,state,pred,sim,z_relative_mob,le,ranking,order,cn
48025,41,35.03455,1,-0.03526187,31.51167,1,1,"Bee, TX"
48025,41,35.73264,1,3.53846431,31.51167,1,2,"Bee, TX"
48025,41,35.17385,2,-0.03526187,31.51167,1,1,"Bee, TX"
48025,41,35.58200,2,3.53846431,31.51167,1,2,"Bee, TX"
48025,41,34.70806,3,-0.03526187,31.51167,1,1,"Bee, TX"
48025,41,35.72634,3,3.53846431,31.51167,1,2,"Bee, TX"


In [498]:
sim_male_bottom[, fcn := fct_rev(factor(cn))]

In [499]:
sim_male_bottom[fcn=='Shelby, IN', le][1]

[1] 32.98131

In [500]:
counties = unique(sim_male_bottom[, county])

q4_male = male[county %in% counties & income_q=='Q4', .(county, le)]
q4_male = merge(q4_male, cn, by = c('county'))

q4_male[, fcn := fct_rev(factor(cn))]
q4_male = q4_male[, .(fcn, le)]

In [1]:
sim_male_bottom

ERROR: Error in eval(expr, envir, enclos): object 'sim_male_bottom' not found


In [501]:
# plot
savepdf('plots/counties_male')
plot = ggplot(sim_male_bottom, aes(y =fcn)) + 
  geom_density_ridges(aes(x = pred, fill = paste(order)), 
           alpha = .5, color = "white", from = 33, to = 39, scale = 0.8) +
  geom_segment(data = q4_male, aes(x = le, xend = le, y = as.numeric(fcn),
                                      yend = as.numeric(fcn) + .9), color='orange', size=0.4, linetype=1) + 
  labs(x = "E(40)",
       y = "") +  theme_ridges(grid=FALSE)+ 
  scale_y_discrete(expand = c(0.01, 0)) +
  scale_x_continuous(expand = c(0.01, 0))  +
  scale_fill_cyclical(values = c("#ff0000", "#0000ff", "#ff8080", "#8080ff"), 
                     guide= 'legend', name='', 
                     labels = c('Observed Income Mobility','Highest Income Mobility')) + 
  theme(legend.position = "top", 
        axis.title=element_text(size=10), 
        axis.text.x = element_text(size=10), 
        axis.text.y = element_text(size=10))
print(plot)
dev.off()

Picking joint bandwidth of 0.0431


pdf 
  2

### Female

In [502]:
max_mob = max(female$z_relative_mob)
max_mob

[1] 3.538464

In [503]:
bottom_female = head(female[income_qr==1, vars, with=FALSE], 10)
bottom_female_c = copy(bottom_female)
bottom_female_c[, z_relative_mob := max_mob]
tot_bottom_female = rbind(bottom_female, bottom_female_c) 
tot_bottom_female[, ranking := .GRP, by = .(county, state)]
tot_bottom_female[, order := seq_len(.N), by = .(county, state)]
anyDuplicated(tot_bottom_female[,.(state, county, z_relative_mob)])

[1] 0

In [504]:
sim_female_bottom = simulate_pred_re(f2_1, nsim=2000, tot_bottom_female, contrast='z_relative_mob', 
                  random_intercept='state')

sim_female_bottom = merge(sim_female_bottom, cn, by = c('county', 'state'))
setorder(sim_female_bottom, ranking)
head(sim_female_bottom)

county,state,pred,sim,z_relative_mob,le,ranking,order,cn
18133,12,40.56340,1,-0.1347668,37.88754,1,1,"Putnam, IN"
18133,12,41.40714,1,3.5384643,37.88754,1,2,"Putnam, IN"
18133,12,40.56808,2,-0.1347668,37.88754,1,1,"Putnam, IN"
18133,12,41.76068,2,3.5384643,37.88754,1,2,"Putnam, IN"
18133,12,40.47165,3,-0.1347668,37.88754,1,1,"Putnam, IN"
18133,12,41.30827,3,3.5384643,37.88754,1,2,"Putnam, IN"


In [505]:
sim_female_bottom[, fcn := fct_rev(factor(cn))]

In [506]:
counties = unique(sim_female_bottom[, county])

q4_female = female[county %in% counties & income_q=='Q4', .(county, le)]
q4_female = merge(q4_female, cn, by = c('county'))

q4_female[, fcn := fct_rev(factor(cn))]

q4_female = q4_female[, .(fcn, le)]

In [507]:
# plot
savepdf('plots/counties_female')
plot = ggplot(sim_female_bottom, aes(y = fcn)) + 
  geom_density_ridges(aes(x = pred, fill = paste(order)), 
           alpha = .5, color = "white", from = 39, to = 44, scale = 1) +
  geom_segment(data = q4_female, aes(x = le, xend = le, y = as.numeric(fcn),
                                      yend = as.numeric(fcn) + .9), color='orange', size=0.4, linetype=1) + 
  labs(x = "E(40)",
       y = "") +  theme_ridges(grid=FALSE)+ 
  scale_y_discrete(expand = c(0.01, 0)) +
  scale_x_continuous(expand = c(0.01, 0))  +
  scale_fill_cyclical(values = c("#ff0000", "#0000ff", "#ff8080", "#8080ff"), 
                     guide= 'legend', name='', 
                     labels = c('Observed Income Mobility','Highest Income Mobility')) + 
  theme(legend.position = "top", 
        axis.title=element_text(size=10), 
        axis.text.x = element_text(size=10), 
        axis.text.y = element_text(size=10))
print(plot)
dev.off()

Picking joint bandwidth of 0.0448


pdf 
  2